In [1]:
# %cd 
from GeogridOpticals import GeogridOptical
# %cd exp2

Using default ISCE Path: /home/ayush12gupta/anaconda3/envs/ISCE2/lib/python3.6/site-packages/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


In [2]:
conda activate isce


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import numpy as np
import pyproj
def loadProduct(xmlname):
    '''
    Load the product using Product Manager.
    '''
    import isce
    from iscesys.Component.ProductManager import ProductManager as PM

    pm = PM()
    pm.configure()

    obj = pm.loadProduct(xmlname)

    return obj


def getMergedOrbit(product):
    import isce
    from isceobj.Orbit.Orbit import Orbit

    ###Create merged orbit
    orb = Orbit()
    orb.configure()

    burst = product[0].bursts[0]
    #Add first burst orbit to begin with
    for sv in burst.orbit:
        orb.addStateVector(sv)


    for pp in product:
        ##Add all state vectors
        for bb in pp.bursts:
            for sv in bb.orbit:
                if (sv.time< orb.minTime) or (sv.time > orb.maxTime):
                    orb.addStateVector(sv)

    return orb

def loadMetadata(indir):
    '''
    Input file.
    '''
    import os
    import numpy as np

    frames = []
    for swath in range(1,4):
        inxml = os.path.join(indir, 'IW{0}.xml'.format(swath))
        if os.path.exists(inxml):
            ifg = loadProduct(inxml)
            frames.append(ifg)

    info =  {}
    info['sensingStart'] = min([x.sensingStart for x in frames])
    info['sensingStop'] = max([x.sensingStop for x in frames])
    info['startingRange'] = min([x.startingRange for x in frames])
    info['farRange'] = max([x.farRange for x in frames])
    info['prf'] = 1.0 / frames[0].bursts[0].azimuthTimeInterval
    info['rangePixelSize'] = frames[0].bursts[0].rangePixelSize
    info['lookSide'] = -1
    info['numberOfLines'] = int( np.round( (info['sensingStop'] - info['sensingStart']).total_seconds() * info['prf'])) + 1
    info['numberOfSamples'] = int( np.round( (info['farRange'] - info['startingRange'])/info['rangePixelSize'])) + 1
    info['orbit'] = getMergedOrbit(frames)

    return info

def gps_to_ecef_pyproj(lat_lon_alt, radians=True):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, lat_lon_alt[0], lat_lon_alt[1], lat_lon_alt[2], radians=radians)
    xyz = np.array([x, y, z])
    return xyz

def gps_to_ecef_llh(xyz, radians=True):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    lat, lon, alt = pyproj.transform(ecef, lla, xyz[0], xyz[1], xyz[2], radians=radians)
    lat_lon_alt = np.array([lat, lon, alt])
    return lat_lon_alt

In [4]:
%cd ../exp2_1

/run/user/1000/gvfs/sftp:host=172.26.126.105/DATA/glacier-vel/exp2_1


In [5]:
metadata_m = loadMetadata('secondary')
metadata_s = loadMetadata('fine_coreg')
# metadata_m['orbit'].

In [6]:
(metadata_m['sensingStart'] - metadata_s['sensingStart']).total_seconds()

1036800.708495

In [9]:
!ls ..

13oct_25oct   dinsar	 feb_pair3		jan_pair1_1  new
19sept_1oct   exp	 geogrid_autorift	jan_pair2    notebooks
1oct_13oct    exp2	 geogrid_req		jan_pair3    tops
9july_19sept  exp2_1	 glacier_flow_analysis	jul_189pair
cmd.txt       feb_pair1  isce2			mar_pair1
data	      feb_pair2  jan_pair1		mar_pair2


In [12]:
from osgeo import gdal

obj = GeogridOptical()
dem_info = gdal.Info('/home/ayush12gupta/Desktop/ugp/exp2/demtest_roi.tif', format='json')
obj.startingRange = metadata_m['startingRange']
obj.rangePixelSize = metadata_m['rangePixelSize']
obj.sensingStart = metadata_m['sensingStart']
obj.prf = metadata_m['prf']
obj.lookSide = metadata_m['lookSide']
obj.repeatTime = (metadata_s['sensingStart'] - metadata_m['sensingStart']).total_seconds()
obj.numberOfLines = metadata_m['numberOfLines']
obj.numberOfSamples = metadata_m['numberOfSamples']
obj.nodata_out = -32767
obj.chipSizeX0 = 240
print(obj.startingRange, obj.rangePixelSize, obj.sensingStart)
obj.gridSpacingX = dem_info['geoTransform'][1]
obj.orbit = metadata_m['orbit']
obj.demname = 'demtest_roi.tif'
obj.dhdxname = 'demtest_roi_X.tif'
obj.dhdyname = 'demtest_roi_Y.tif'
obj.vxname = ''
obj.vyname = ''
obj.srxname = ''
obj.sryname = ''
obj.csminxname = ''
obj.csminyname = ''
obj.csmaxxname = ''
obj.csmaxyname = ''
obj.ssmname = 'ssm.tif'
obj.winlocname = "window_location.tif"
obj.winoffname = "window_offset.tif"
obj.winsrname = "window_search_range.tif"
obj.wincsminname = "window_chip_size_min.tif"
obj.wincsmaxname = "window_chip_size_max.tif"
obj.winssmname = "window_stable_surface_mask.tif"
obj.winro2vxname = "window_rdr_off2vel_x_vec.tif"
obj.winro2vyname = "window_rdr_off2vel_y_vec.tif"

obj.getIncidenceAngle()
obj.runGeogrid()

846099.1914484155 2.329562114715323 2019-07-13 12:56:16.215079
demtest_roi.tif


AttributeError: 'NoneType' object has no attribute 'GetProjection'

In [49]:
# (np.floor(obj.numberOfLines/2)-1) / obj.prf
obj.startingRange

846099.1914484155

In [11]:
import numpy as np
from osgeo import osr
from iscesys import DateTimeUtil as DTU
import datetime
import struct

grd_res = obj.rangePixelSize / math.sin(obj.incidenceAngle)
sensingStart = DTU.seconds_since_midnight(obj.sensingStart)
tmid = obj.sensingStart + datetime.timedelta(seconds = (np.floor(obj.numberOfLines/2)-1) / obj.prf)
satvmid = obj.orbit.interpolateOrbit(tmid)._velocity
satxmid = obj.orbit.interpolateOrbit(tmid)._position
azm_res = np.linalg.norm(satvmid) / obj.prf
ChipSizeX0 = obj.chipSizeX0
ChipSizeX0_PIX_grd = np.ceil(ChipSizeX0 / grd_res / 4) * 4;
ChipSizeX0_PIX_azm = np.ceil(ChipSizeX0 / azm_res / 4) * 4;
print("Ground range pixel size: ", grd_res)
print("Azimuth pixel size: ", azm_res)
print("Incidence Angle: ", obj.incidenceAngle*180/np.pi)

print("Map inputs: ")
print("EPSG: " + str(obj.epsgDem))
print("Smallest Allowable Chip Size in m: " + str(obj.chipSizeX0))
print("Repeat Time: " + str(obj.repeatTime))
print("XLimits: " + str(obj._xlim[0]) + "  " + str(obj._xlim[1]))
print("YLimits: " + str(obj._ylim[0]) + "  " + str(obj._ylim[1]))
print("Extent in km: " + str((obj._xlim[1]-obj._xlim[0])/1000.0) + "  " + str((obj._ylim[1]-obj._ylim[0])/1000.0))
print("Output Nodata Value: " + str(obj.nodata_out) + "\n")

demDS = gdal.Open(obj.demname, gdal.GA_ReadOnly)

if (obj.dhdxname != ""):
    sxDS = gdal.Open(obj.dhdxname, gdal.GA_ReadOnly)
    syDS = gdal.Open(obj.dhdyname, gdal.GA_ReadOnly)

if (obj.vxname != ""):
    vxDS = gdal.Open(obj.vxname, gdal.GA_ReadOnly)
    vyDS = gdal.Open(obj.vyname, gdal.GA_ReadOnly)

if (obj.srxname != ""):
    srxDS = gdal.Open(obj.srxname, gdal.GA_ReadOnly)
    sryDS = gdal.Open(obj.sryname, gdal.GA_ReadOnly)

if (obj.csminxname != ""):
    csminxDS = gdal.Open(obj.csminxname, gdal.GA_ReadOnly)
    csminyDS = gdal.Open(obj.csminyname, gdal.GA_ReadOnly)

if (obj.csmaxxname != ""):
    csmaxxDS = gdal.Open(obj.csmaxxname, gdal.GA_ReadOnly)
    csmaxyDS = gdal.Open(obj.csmaxyname, gdal.GA_ReadOnly)

if (obj.ssmname != ""):
    ssmDS = gdal.Open(obj.ssmname, gdal.GA_ReadOnly)

geoTrans = demDS.GetGeoTransform()
demXSize = demDS.RasterXSize
demYSize = demDS.RasterYSize

lOff = int(np.max( [np.floor((obj._ylim[1] - geoTrans[3])/geoTrans[5]), 0.]))
lCount = int(np.min([ np.ceil((obj._ylim[0] - geoTrans[3])/geoTrans[5]), demYSize-1.]) - lOff)

pOff = int(np.max([ np.floor((obj._xlim[0] - geoTrans[0])/geoTrans[1]), 0.]))
print( np.ceil((obj._xlim[1] - geoTrans[0])/geoTrans[1]), demXSize-1.)
pCount = int(np.min([ np.ceil((obj._xlim[1] - geoTrans[0])/geoTrans[1]), demXSize-1.]) - pOff)

print(pOff, pCount, lOff, lCount)

print("Xlimits : " + str(geoTrans[0] + pOff * geoTrans[1]) +  "  " + str(geoTrans[0] + (pOff + pCount) * geoTrans[1]))

print("Ylimits : " + str(geoTrans[3] + (lOff + lCount) * geoTrans[5]) +  "  " + str(geoTrans[3] + lOff * geoTrans[5]))

print("Origin index (in DEM) of geogrid: " + str(pOff) + "   " + str(lOff))
obj.pOff = pOff
obj.lOff = lOff

print("Dimensions of geogrid: " + str(pCount) + " x " + str(lCount))
obj.pCount = pCount
obj.lCount = lCount

#--------------------------------------------------------------------------------------------

projDem = osr.SpatialReference()
if obj.epsgDem:
    projDem.ImportFromEPSG(obj.epsgDem)

projDat = osr.SpatialReference()
projDat.ImportFromEPSG(4326)
    
fwdTrans = osr.CoordinateTransformation(projDem, projDat)
invTrans = osr.CoordinateTransformation(projDat, projDem)

if (obj.vxname != ""):
    nodata = vxDS.GetRasterBand(1).GetNoDataValue()
else:
    nodata = 0

nodata_out = obj.nodata_out

pszFormat = "GTiff"
adfGeoTransform = ( geoTrans[0] + pOff * geoTrans[1], geoTrans[1], 0, geoTrans[3] + lOff * geoTrans[5], 0, geoTrans[5] )
oSRS = osr.SpatialReference()
pszSRS_WKT = projDem.ExportToWkt()

poDriver = gdal.GetDriverByName(pszFormat)
if( poDriver is None ):
    raise Exception('Cannot create gdal driver for output')

pszDstFilename = obj.winlocname
poDstDS = poDriver.Create(pszDstFilename, xsize=pCount, ysize=lCount, bands=2, eType=gdal.GDT_Int32)
poDstDS.SetGeoTransform( adfGeoTransform )
poDstDS.SetProjection( pszSRS_WKT )

poBand1 = poDstDS.GetRasterBand(1)
poBand2 = poDstDS.GetRasterBand(2)
poBand1.SetNoDataValue(nodata_out)
poBand2.SetNoDataValue(nodata_out)

if (obj.dhdxname != ""):
    poDriverRO2VX = gdal.GetDriverByName(pszFormat)
    if( poDriverRO2VX is None ):
        raise Exception('Cannot create gdal driver for output')

    pszDstFilenameRO2VX = obj.winro2vxname
    poDstDSRO2VX = poDriverRO2VX.Create(pszDstFilenameRO2VX, xsize=pCount, ysize=lCount, bands=2, eType=gdal.GDT_Float64)
    poDstDSRO2VX.SetGeoTransform( adfGeoTransform )
    poDstDSRO2VX.SetProjection( pszSRS_WKT )

    poBand1RO2VX = poDstDSRO2VX.GetRasterBand(1)
    poBand2RO2VX = poDstDSRO2VX.GetRasterBand(2)
    poBand1RO2VX.SetNoDataValue(nodata_out)
    poBand2RO2VX.SetNoDataValue(nodata_out)
    
    poDriverRO2VY = gdal.GetDriverByName(pszFormat)
    if( poDriverRO2VY is None ):
        raise Exception('Cannot create gdal driver for output')

    pszDstFilenameRO2VY = obj.winro2vyname
    poDstDSRO2VY = poDriverRO2VY.Create(pszDstFilenameRO2VY, xsize=pCount, ysize=lCount, bands=2, eType=gdal.GDT_Float64)
    poDstDSRO2VY.SetGeoTransform( adfGeoTransform )
    poDstDSRO2VY.SetProjection( pszSRS_WKT )

    poBand1RO2VY = poDstDSRO2VY.GetRasterBand(1)
    poBand2RO2VY = poDstDSRO2VY.GetRasterBand(2)
    poBand1RO2VY.SetNoDataValue(nodata_out)
    poBand2RO2VY.SetNoDataValue(nodata_out)

raster1 = np.zeros(pCount,dtype=np.int32)
raster2 = np.zeros(pCount,dtype=np.int32)
raster11 = np.zeros(pCount,dtype=np.int32)
raster22 = np.zeros(pCount,dtype=np.int32)
sr_raster11 = np.zeros(pCount,dtype=np.int32)
sr_raster22 = np.zeros(pCount,dtype=np.int32)
csmin_raster11 = np.zeros(pCount,dtype=np.int32)
csmin_raster22 = np.zeros(pCount,dtype=np.int32)
csmax_raster11 = np.zeros(pCount,dtype=np.int32)
csmax_raster22 = np.zeros(pCount,dtype=np.int32)
ssm_raster = np.zeros(pCount,dtype=np.int32)
raster1a = np.zeros(pCount,dtype=np.float64)
raster1b = np.zeros(pCount,dtype=np.float64)
raster1c = np.zeros(pCount,dtype=np.float64)
raster2a = np.zeros(pCount,dtype=np.float64)
raster2b = np.zeros(pCount,dtype=np.float64)
raster2c = np.zeros(pCount,dtype=np.float64)

#   X- and Y-direction pixel size
# X_res = np.abs(obj.XSize)
# Y_res = np.abs(obj.YSize)
# print("X-direction pixel size: " + str(X_res))
# print("Y-direction pixel size: " + str(Y_res))
# obj.X_res = X_res
# obj.Y_res = Y_res

# ChipSizeX0_PIX_X = np.ceil(self.chipSizeX0 / X_res / 4) * 4
# ChipSizeX0_PIX_Y = np.ceil(self.chipSizeX0 / Y_res / 4) * 4

Ground range pixel size:  3.4891158788638763
Azimuth pixel size:  15.612188211132246
Incidence Angle:  41.88713762241911
Map inputs: 
EPSG: None
Smallest Allowable Chip Size in m: 240
Repeat Time: -1036800.708495


TypeError: 'NoneType' object is not subscriptable

In [10]:
# math.sin(obj.incidenceAngle)
# projDem.ExportFromWkt()
obj.lookSide
# llh = np.array([0.5, 0.15, 0.6])
# llh[:2]=llh[:2]*np.pi/180.0
# llh

-1

In [18]:
deg2rad = np.pi/180.0
wgs84_e2 = 0.0066943799901
for ii in range(1):  #lCount):
    y = geoTrans[3] + (lOff+ii+0.5) * geoTrans[5]
    demLine = demDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
    demLine = struct.unpack('d' * pCount, demLine)

    if (obj.dhdxname != ""):
        sxLine = sxDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        sxLine = struct.unpack('d' * pCount, sxLine)
        syLine = syDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        syLine = struct.unpack('d' * pCount, syLine)

    if (obj.vxname != ""):
        vxLine = vxDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        vxLine = struct.unpack('d' * pCount, vxLine)
        vyLine = vyDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        vyLine = struct.unpack('d' * pCount, vyLine)

    if (obj.ssmname != ""):
        ssmLine = ssmDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        ssmLine = struct.unpack('d' * pCount, ssmLine)

    for jj in range(1):  #pCount):
        llh = np.array([geoTrans[0] + (jj+pOff+0.5)*geoTrans[1], y, demLine[jj]])
        targllh0 = llh.copy()
        if (obj.dhdxname != ""):
            slp = np.array([sxLine[jj], syLine[jj], -1.0])
        if (obj.vxname != ""):
            vel = np.array([vxLine[jj], vyLine[jj], 0.0])
        else:
            vel = np.array([0., 0., 0.])
        llh = np.array(fwdTrans.TransformPoint(llh[0],llh[1],llh[2]))
        llhi = llh.copy()
        llhi[:2] = deg2rad*llh[:2]
        
        xyz = np.array(gps_to_ecef_pyproj(llhi))
        
        satx = satxmid.copy()
        satv = satvmid.copy()
        
        tline = tmid
        for i in range(51):
            tprev = tline
            drpos = xyz - satx
            rngpix = np.linalg.norm(drpos)
            fn = np.dot(drpos, satv)
            fnprime = -1*np.dot(satv, satv)
            tline = tline - datetime.timedelta(seconds = (fn/fnprime))
            satv = obj.orbit.interpolateOrbit(tline)._velocity
            satx = obj.orbit.interpolateOrbit(tline)._position
        
        rgind = np.round((rngpix - obj.startingRange) / obj.rangePixelSize) + 0.;
        azind = np.round((tline - obj.sensingStart).total_seconds() * obj.prf) + 0.;
        
        los = drpos / np.linalg.norm(drpos)
        llh = xyz + los*obj.rangePixelSize
        llhi = np.array(gps_to_ecef_llh(llh))
        
        llh[:2] = llhi[:2] / deg2rad
        llh[2] = llhi[2];
        
        llh = np.array(invTrans.TransformPoint(llh[0],llh[1],llh[2]))
        drpos =  llh - targllh0
        los = drpos / np.linalg.norm(drpos)
        
        tline = tline + datetime.timedelta(seconds=1/obj.prf)
        satv = np.array(obj.orbit.interpolateOrbit(tline)._velocity)
        satx = np.array(obj.orbit.interpolateOrbit(tline)._position)
        
        dopfact = 0.0;
        height = demLine[jj]
        vhat = satv / np.linalg.norm(satv)
        vmag = np.linalg.norm(satv)

        # Convert position and velocity to local tangent plane
        major = 6378137.0
        minor = major * math.sqrt(1 - wgs84_e2)

        # Setup ortho normal system right below satellite
        satDist = np.linalg.norm(satx)
        temp = np.array([(satx[0] / major), (satx[1] / major), (satx[2] / minor)])
        alpha = 1 / np.linalg.norm(temp)
        radius = alpha * satDist
        hgt = (1.0 - alpha) * satDist

        # Setup TCN basis - Geocentric
        nhat = -1*satx / np.linalg.norm(satx)
        
        temp = np.cross(nhat,satv)
        chat = temp / np.linalg.norm(temp)
        temp = np.cross(chat,nhat)
        that = temp / np.linalg.norm(temp)


        # Solve the range doppler eqns iteratively
        # Initial guess
        zsch = height;
        
        for i in range(10):
            a = satDist
            b = (radius + zsch)

            costheta = 0.5 * (a / rngpix + rngpix / a - (b / a) * (b / rngpix))
            sintheta = math.sqrt(1-costheta*costheta)

            gamma = rngpix * costheta
            alpha = dopfact - gamma * np.dot(nhat,vhat) / np.dot(vhat,that)
            beta = -obj.lookSide * math.sqrt(rngpix * rngpix * sintheta * sintheta - alpha * alpha);
            delta = alpha*that + beta*chat + gamma*nhat
            targVec = satx + delta
            
            # latlon_C(&wgs84, targVec, llhi, XYZ_2_LLH);
            llhi = gps_to_ecef_llh(targVec)
            llhi[2] = height;
            # latlon_C(&wgs84, targXYZ, llhi, LLH_2_XYZ);
            targXYZ = gps_to_ecef_pyproj(llhi)

            zsch = np.linalg.norm(targXYZ) - radius
            diffvec = satx - targXYZ
            rdiff  = rngpix - np.linalg.norm(diffvec)
        
        llh[:2] = llhi[:2] / deg2rad;
        llh[2] = llhi[2]
        llh = np.array(invTrans.TransformPoint(llh[0],llh[1],llh[2]))
        alt = llh - targllh0
        temp = alt / np.linalg.norm(alt)
        if (obj.dhdxname != ""):
            normal = slp / -1*np.linalg.norm(slp)
        else:
            normal = np.array([0., 0., 0.])
        if (obj.vxname != ""):
            vel[2] = -(vel[0]*normal[0]+vel[1]*normal[1])/normal[2]
        

        if (rgind > obj.numberOfSamples-1) or (rgind < 0) or (azind > obj.numberOfLines-1) or (azind < 0):
            raster1[jj] = nodata_out
            raster2[jj] = nodata_out
            raster11[jj] = nodata_out
            raster22[jj] = nodata_out

            sr_raster11[jj] = nodata_out
            sr_raster22[jj] = nodata_out
            csmin_raster11[jj] = nodata_out
            csmin_raster22[jj] = nodata_out
            csmax_raster11[jj] = nodata_out
            csmax_raster22[jj] = nodata_out
            ssm_raster[jj] = nodata_out

            raster1a[jj] = nodata_out
            raster1b[jj] = nodata_out
            raster1c[jj] = nodata_out
            raster2a[jj] = nodata_out
            raster2b[jj] = nodata_out
            raster2c[jj] = nodata_out
            
            
        else:
            raster1[jj] = rgind
            raster2[jj] = azind

            if (obj.dhdxname != ""):
                if (obj.vxname != ""):
                    if (vel[0] == nodata):
                        raster11[jj] = 0.
                        raster22[jj] = 0.
                    else:
                        raster11[jj] = np.round(np.dot(vel,los)*obj.repeatTime/obj.rangePixelSize/365.0/24.0/3600.0*1)
                        raster22[jj] = np.round(np.dot(vel,yunit)*obj.repeatTime/np.linalg.norm(alt)/365.0/24.0/3600.0*1)

                cross = np.cross(los,temp)
                cross = cross / np.linalg.norm(cross)
                cross_check = np.abs(np.arccos(np.dot(normal,cross))/deg2rad-90.0)
                print(cross_check)

                if (cross_check > 1.0):
                    raster1a[jj] = normal[2]/(obj.repeatTime/self.XSize/365.0/24.0/3600.0)*(normal[2]*yunit[1]-normal[1]*yunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster1b[jj] = -normal[2]/(obj.repeatTime/self.YSize/365.0/24.0/3600.0)*(normal[2]*xunit[1]-normal[1]*xunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster2a[jj] = -normal[2]/(obj.repeatTime/self.XSize/365.0/24.0/3600.0)*(normal[2]*yunit[0]-normal[0]*yunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster2b[jj] = normal[2]/(obj.repeatTime/self.YSize/365.0/24.0/3600.0)*(normal[2]*xunit[0]-normal[0]*xunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                else:
                    raster1a[jj] = nodata_out
                    raster1b[jj] = nodata_out
                    raster2a[jj] = nodata_out
                    raster2b[jj] = nodata_out

#             if (self.ssmname != ""):
#                 ssm_raster[jj] = ssmLine[jj]


42.692251038054145


In [17]:
obj.repeatTime

0.0

In [35]:
# llh = [76.86643125, 32.78370023,  0.]
# deg2rad
llh[:2] = llh[:2]*deg2rad
llh
# np.pi/180
#         xind = np.round((targutm0[0] - self.startingX) / self.XSize) + 1.
#         yind = np.round((targutm0[1] - self.startingY) / self.YSize) + 1.

#         #   x-direction vector
#         targutm = targutm0.copy()
#         targutm[0] = targutm0[0] + self.XSize
#         targxyz = np.array(invTrans.TransformPoint(targutm[0],targutm[1],targutm[2]))
#         xunit = (targxyz-targxyz0) / np.linalg.norm(targxyz-targxyz0)

#         #   y-direction vector
#         targutm = targutm0.copy()
#         targutm[1] = targutm0[1] + self.YSize
#         targxyz = np.array(invTrans.TransformPoint(targutm[0],targutm[1],targutm[2]))
#         yunit = (targxyz-targxyz0) / np.linalg.norm(targxyz-targxyz0)

array([1.34157231, 0.57218351, 0.        ])

In [46]:
import numpy as np
import math

def gps_to_ecef_custom(lat, lon, alt):
    rad_lat = lat * (math.pi / 180.0)
    rad_lon = lon * (math.pi / 180.0)

    a = 6378137.0
    finv = 298.257223563
    f = 1 / finv
    e2 = 1 - (1 - f) * (1 - f)
    v = a / math.sqrt(1 - e2 * math.sin(rad_lat) * math.sin(rad_lat))

    x = (v + alt) * math.cos(rad_lat) * math.cos(rad_lon)
    y = (v + alt) * math.cos(rad_lat) * math.sin(rad_lon)
    z = (v * (1 - e2) + alt) * math.sin(rad_lat)

    return x, y, z



pt = [1.34157231, 0.57218351, 0.]
llh = [76.86643125, 32.78370023,  0.]
xyz = [1219613.96504615, 5227103.99214356, 3433815.83688019]
# for pt in coords:
print('pyproj', gps_to_ecef_llh(xyz, True))
# print('pyproj', gps_to_ecef_custom(llh[1], llh[0], llh[2]))

# runGeogrid(metadata_m, metadata_s, inps.demfile, inps.dhdxfile, inps.dhdyfile, inps.vxfile, inps.vyfile, inps.srxfile, inps.sryfile, inps.csminxfile, inps.csminyfile, inps.csmaxxfile, inps.csmaxyfile, inps.ssmfile)
# obj.incidenceAngle

# XLimits: 567375.1938239019  760887.9992762555
# YLimits: 3503174.9300490217  3648544.2879937193
# Extent in km: 193.51280545235355  145.3693579446976
# Output Nodata Value: -32767

# Xlimits : 674777.4423958672  760890.8205813951
# Ylimits : 3549293.760153323  3628864.074188244
# Origin index (in DEM) of geogrid: 0   0
# Dimensions of geogrid: 3106 x 2870

pyproj [1.34157231e+00 5.72183510e-01 3.72529030e-09]


In [ ]:
demDS = gdal.Open(self.demname, gdal.GA_ReadOnly)

if (self.dhdxname != ""):
    sxDS = gdal.Open(self.dhdxname, gdal.GA_ReadOnly)
    syDS = gdal.Open(self.dhdyname, gdal.GA_ReadOnly)
    
geoTrans = demDS.GetGeoTransform()
demXSize = demDS.RasterXSize
demYSize = demDS.RasterYSize


In [ ]:
if (self.dhdxname != ""):
    poDriverRO2VX = gdal.GetDriverByName(pszFormat)
    if( poDriverRO2VX is None ):
        raise Exception('Cannot create gdal driver for output')

    pszDstFilenameRO2VX = self.winro2vxname
    poDstDSRO2VX = poDriverRO2VX.Create(pszDstFilenameRO2VX, xsize=pCount, ysize=lCount, bands=2, eType=gdal.GDT_Float64)
    poDstDSRO2VX.SetGeoTransform( adfGeoTransform )
    poDstDSRO2VX.SetProjection( pszSRS_WKT )

    poBand1RO2VX = poDstDSRO2VX.GetRasterBand(1)
    poBand2RO2VX = poDstDSRO2VX.GetRasterBand(2)
    poBand1RO2VX.SetNoDataValue(nodata_out)
    poBand2RO2VX.SetNoDataValue(nodata_out)
    
    poDriverRO2VY = gdal.GetDriverByName(pszFormat)
    if( poDriverRO2VY is None ):
        raise Exception('Cannot create gdal driver for output')

    pszDstFilenameRO2VY = self.winro2vyname
    poDstDSRO2VY = poDriverRO2VY.Create(pszDstFilenameRO2VY, xsize=pCount, ysize=lCount, bands=2, eType=gdal.GDT_Float64)
    poDstDSRO2VY.SetGeoTransform( adfGeoTransform )
    poDstDSRO2VY.SetProjection( pszSRS_WKT )

    poBand1RO2VY = poDstDSRO2VY.GetRasterBand(1)
    poBand2RO2VY = poDstDSRO2VY.GetRasterBand(2)
    poBand1RO2VY.SetNoDataValue(nodata_out)
    poBand2RO2VY.SetNoDataValue(nodata_out)

In [ ]:
for ii in range(lCount):
    y = geoTrans[3] + (lOff+ii+0.5) * geoTrans[5]
    demLine = demDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
    demLine = struct.unpack('d' * pCount, demLine)

    if (self.dhdxname != ""):
        sxLine = sxDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        sxLine = struct.unpack('d' * pCount, sxLine)
        syLine = syDS.GetRasterBand(1).ReadRaster(xoff=pOff, yoff=lOff+ii, xsize=pCount, ysize=1, buf_xsize=pCount, buf_ysize=1, buf_type=gdal.GDT_Float64)
        syLine = struct.unpack('d' * pCount, syLine)

    

    for jj in range(pCount):
        xyzs = np.array([geoTrans[0] + (jj+pOff+0.5)*geoTrans[1], y, demLine[jj]])
        targxyz0 = xyzs.copy()
        if (self.dhdxname != ""):
            slp = np.array([sxLine[jj], syLine[jj], -1.0])
        if (self.vxname != ""):
            vel = np.array([vxLine[jj], vyLine[jj], 0.0])
        else:
            vel = np.array([0., 0., 0.])
        if (self.srxname != ""):
            schrng1 = np.array([srxLine[jj], sryLine[jj], 0.0])
            schrng2 = np.array([-srxLine[jj], sryLine[jj], 0.0])
        targutm0 = np.array(fwdTrans.TransformPoint(targxyz0[0],targxyz0[1],targxyz0[2]))
        xind = np.round((targutm0[0] - self.startingX) / self.XSize) + 1.
        yind = np.round((targutm0[1] - self.startingY) / self.YSize) + 1.

        #   x-direction vector
        targutm = targutm0.copy()
        targutm[0] = targutm0[0] + self.XSize
        targxyz = np.array(invTrans.TransformPoint(targutm[0],targutm[1],targutm[2]))
        xunit = (targxyz-targxyz0) / np.linalg.norm(targxyz-targxyz0)

        #   y-direction vector
        targutm = targutm0.copy()
        targutm[1] = targutm0[1] + self.YSize
        targxyz = np.array(invTrans.TransformPoint(targutm[0],targutm[1],targutm[2]))
        yunit = (targxyz-targxyz0) / np.linalg.norm(targxyz-targxyz0)

        #   local normal vector
        if (self.dhdxname != ""):
            normal = -slp / np.linalg.norm(slp)
        else:
            normal = np.array([0., 0., 0.])


        if ((xind > self.numberOfSamples)|(xind < 1)|(yind > self.numberOfLines)|(yind < 1)):
#                    pdb.set_trace()
            raster1[jj] = nodata_out
            raster2[jj] = nodata_out
            raster11[jj] = nodata_out
            raster22[jj] = nodata_out

            sr_raster11[jj] = nodata_out
            sr_raster22[jj] = nodata_out
            csmin_raster11[jj] = nodata_out
            csmin_raster22[jj] = nodata_out
            csmax_raster11[jj] = nodata_out
            csmax_raster22[jj] = nodata_out
            ssm_raster[jj] = nodata_out

            raster1a[jj] = nodata_out
            raster1b[jj] = nodata_out
            raster2a[jj] = nodata_out
            raster2b[jj] = nodata_out
        else:
            raster1[jj] = xind;
            raster2[jj] = yind;

            if (self.dhdxname != ""):

            
                cross = np.cross(xunit,yunit)
                cross = cross / np.linalg.norm(cross)
                cross_check = np.abs(np.arccos(np.dot(normal,cross))/np.pi*180.0-90.0)

                if (cross_check > 1.0):
                    raster1a[jj] = normal[2]/(self.repeatTime/self.XSize/365.0/24.0/3600.0)*(normal[2]*yunit[1]-normal[1]*yunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster1b[jj] = -normal[2]/(self.repeatTime/self.YSize/365.0/24.0/3600.0)*(normal[2]*xunit[1]-normal[1]*xunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster2a[jj] = -normal[2]/(self.repeatTime/self.XSize/365.0/24.0/3600.0)*(normal[2]*yunit[0]-normal[0]*yunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                    raster2b[jj] = normal[2]/(self.repeatTime/self.YSize/365.0/24.0/3600.0)*(normal[2]*xunit[0]-normal[0]*xunit[2])/((normal[2]*xunit[0]-normal[0]*xunit[2])*(normal[2]*yunit[1]-normal[1]*yunit[2])-(normal[2]*yunit[0]-normal[0]*yunit[2])*(normal[2]*xunit[1]-normal[1]*xunit[2]));
                else:
                    raster1a[jj] = nodata_out
                    raster1b[jj] = nodata_out
                    raster2a[jj] = nodata_out
                    raster2b[jj] = nodata_out